In [2]:
# Load the GEO Data 

In [1]:
import GEOparse
import pandas as pd
import sklearn.cluster

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import numpy as np
from sklearn.cluster import KMeans

In [5]:
gse = GEOparse.get_GEO(filepath="./GSE10245_family.soft.gz")

26-Nov-2022 21:16:01 INFO GEOparse - Parsing ./GSE10245_family.soft.gz: 
26-Nov-2022 21:16:01 DEBUG GEOparse - DATABASE: GeoMiame
26-Nov-2022 21:16:01 DEBUG GEOparse - SERIES: GSE10245
26-Nov-2022 21:16:01 DEBUG GEOparse - PLATFORM: GPL570
/opt/anaconda3/lib/python3.9/site-packages/GEOparse/GEOparse.py:401: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  return read_csv(StringIO(data), index_col=None, sep="\t")
26-Nov-2022 21:16:02 DEBUG GEOparse - SAMPLE: GSM258551
26-Nov-2022 21:16:02 DEBUG GEOparse - SAMPLE: GSM258552
26-Nov-2022 21:16:02 DEBUG GEOparse - SAMPLE: GSM258553
26-Nov-2022 21:16:02 DEBUG GEOparse - SAMPLE: GSM258554
26-Nov-2022 21:16:02 DEBUG GEOparse - SAMPLE: GSM258555
26-Nov-2022 21:16:02 DEBUG GEOparse - SAMPLE: GSM258556
26-Nov-2022 21:16:03 DEBUG GEOparse - SAMPLE: GSM258557
26-Nov-2022 21:16:03 DEBUG GEOparse - SAMPLE: GSM258558
26-Nov-2022 21:16:03 DEBUG GEOparse - SAMPLE: GSM258559
26-Nov-2022 21:16:03 DEBUG 

In [636]:
matrix_data = gse.pivot_samples("VALUE").T

In [637]:
matrix_data

ID_REF,1007_s_at,1053_at,117_at,121_at,1255_g_at,1294_at,1316_at,1320_at,1405_i_at,1431_at,...,AFFX-r2-Bs-thr-M_s_at,AFFX-r2-Ec-bioB-3_at,AFFX-r2-Ec-bioB-5_at,AFFX-r2-Ec-bioB-M_at,AFFX-r2-Ec-bioC-3_at,AFFX-r2-Ec-bioC-5_at,AFFX-r2-Ec-bioD-3_at,AFFX-r2-Ec-bioD-5_at,AFFX-r2-P1-cre-3_at,AFFX-r2-P1-cre-5_at
name,,,,,,,,,,,,,,,,,,,,,
GSM258551,9.129905,8.034022,3.564520,4.746490,2.320698,5.519153,3.339182,2.775395,8.303437,2.981327,...,10.297589,8.857847,9.369037,9.435077,10.936404,10.730983,13.599488,13.031726,15.028729,14.586347
GSM258552,9.843349,7.973332,4.994852,5.197306,2.248520,5.081258,2.934516,2.617097,9.145519,4.619668,...,8.217455,7.865127,8.116953,8.306683,9.986505,9.719266,12.847711,12.250033,14.440756,14.072366
GSM258553,9.730661,8.834045,5.066018,5.234618,2.259504,4.657257,3.007192,2.634559,9.159054,2.746464,...,8.713199,6.996104,7.803869,8.029131,9.468096,9.101115,12.384142,11.798363,14.439887,14.011392
GSM258554,9.032165,7.723965,4.958580,6.078180,2.262787,4.535683,3.167112,3.127495,7.770354,6.212399,...,8.805233,8.221659,8.434090,8.609512,10.064087,9.763076,12.969199,12.307684,14.557363,14.162145
GSM258555,10.281793,9.040800,4.951835,5.205632,2.207531,3.731919,2.711790,2.707079,6.380928,4.528499,...,8.285092,8.042156,8.174685,8.620098,10.005558,9.651785,12.924465,12.243207,14.612223,14.122751
GSM258556,9.154551,8.792375,6.428274,5.009848,2.322618,4.964672,2.958602,2.598406,8.149710,4.008140,...,8.903710,8.088932,8.438776,8.687646,10.212622,9.716659,12.996885,12.369919,14.593751,14.202961
GSM258557,8.063788,8.093354,7.364676,5.533000,2.656929,5.794959,2.984828,2.634927,9.420926,4.039786,...,8.431760,8.395615,8.895953,9.058438,10.523917,10.224866,13.180944,12.688711,14.697993,14.339175
GSM258558,8.881805,7.575620,4.659326,5.362637,2.255955,5.416322,3.075674,2.637682,9.460713,2.797617,...,8.941418,8.205632,8.598923,8.832244,10.313588,9.968033,13.074864,12.541047,14.626500,14.292254
GSM258559,9.039005,8.438594,6.643723,5.210988,2.266399,5.627705,2.975612,2.663941,7.842213,2.748887,...,8.518448,8.189411,8.400951,8.484782,9.954478,9.698518,12.782987,12.103870,14.546733,14.058803


In [646]:
charac_list = []
for gsm in gse.gsms.values():
    charac_list.append(gsm.metadata["characteristics_ch1"][0].split(":")[1][1:])

In [647]:
# Below are part 4,5,6

In [648]:
# Accuracy of training data with model trained on training data 
def compute_accuracy(Train_data,Train_subtype,random_state_num):
    
    #develope the kmeans model
    kmeans = KMeans(n_clusters = 2, random_state = random_state_num).fit(Train_data)
    
    # Predict label(in the form of array)
    predict_label = kmeans.labels_
    # convert predict label array into list 
    predict_label = predict_label.tolist()
    
    # Create a dataframe for predict label and subtype
    df = pd.DataFrame([predict_label,Train_subtype]).T
    df.columns = ["Cluster", "Subtype"]
    
    # Retrive the subtype with cluster == 1
    df_1 = df.loc[df['Cluster'] == 1]
    # Find the most frequent subtype 
    sub_1 = df_1['Subtype'].mode().iloc[0]
    
    # Retrive the subtype with cluster == 0
    df_0 = df.loc[df['Cluster'] == 0]
    # Find the most frequent subtype 
    sub_0 = df_0['Subtype'].mode().iloc[0]
    
    # Define the subtype and corresponding labels
    cancer_subtype = {sub_0: 0,sub_1: 1}
    
    # convert train_cancer_type into train_label
    train_label = [cancer_subtype[item] for item in Train_subtype]
    # Compute accuracy_score 
    return accuracy_score(train_label, predict_label)

In [649]:
# Accuracy of the model trained on all data 
compute_accuracy(matrix_data,charac_list,0)

0.8793103448275862

In [650]:
# Accuracy of the model trained on 50% of training data 
compute_accuracy(X_train,y_train,0)

0.9655172413793104

In [651]:
# Accuracy of testing data with model trained on training data 
def compute_test_accuracy(Data,subtype,random_state_num):
    
    # Split all data into 50% train and 50% test
    X_train, X_test, y_train, y_test = train_test_split(matrix_data,charac_list,test_size=0.5)

    # Develop kmeans model trained based on training data 
    kmeans = KMeans(n_clusters = 2, random_state = random_state_num).fit(X_train)

    # Predict label for testing data
    predict_label = kmeans.fit_predict(X_test)
    # convert predict label array into list 
    predict_label = predict_label.tolist()

    # Create a dataframe for predict label and subtype(test label)
    df = pd.DataFrame([predict_label,y_test]).T
    df.columns = ["Cluster", "Subtype"]

    # Retrive the subtype with cluster == 1
    df_1 = df.loc[df['Cluster'] == 1]
    # Find the most frequent subtype 
    sub_1 = df_1['Subtype'].mode().iloc[0]

    # Retrive the subtype with cluster == 0
    df_0 = df.loc[df['Cluster'] == 0]
    # Find the most frequent subtype 
    sub_0 = df_0['Subtype'].mode().iloc[0]

    # Define the subtype and corresponding labels
    cancer_subtype = {sub_0: 0,sub_1: 1}

    # convert train_cancer_type into train_label
    train_label = [cancer_subtype[item] for item in y_test]
    
    return accuracy_score(train_label, predict_label)

In [653]:
compute_test_accuracy(matrix_data,charac_list,0)

0.9310344827586207